In [12]:
import os
import sys
from keras.preprocessing import image
import numpy as np
import re
from PIL import Image

current_path = os.path.abspath('')
image_path = os.path.join(current_path, "test_img")
input_img_path = os.path.join(current_path, "input_img")
target_img_path = os.path.join(current_path, "target_img")


"""
이미지를 재단하는 함수
입력용 회색이미지, 출력용 색상이미지 재단
"""
def crop_img(image_path):
    
    
    files = os.listdir(image_path)
    select = re.compile("([a-zA-Z0-9_\s\.]+(?=\_pic))")   
   
    for file in files:
        if select.match(str(file)) != None:
            name = str(file)
            os.chdir(image_path)
            img = Image.open(file)

            img = img.crop([136, 112, 917, 823])

            os.chdir(input_img_path)
            img.save(name, "PNG")
            os.chdir('..')
        else:
            name = str(file)
            os.chdir(image_path)
            img = Image.open(file)

            img = img.crop([136, 112, 917, 823])

            os.chdir(target_img_path)
            img.save(name, "PNG")
            os.chdir('..')

crop_img(image_path)

In [13]:
"""
회색이미지를 흑백이미지로 변환하는 함수

"""
import os
import cv2
current_path = os.path.abspath('')
binary_img_path = os.path.join(current_path,"binary")
modified_path = os.path.join(current_path, "input_img")
def img_preprocessing_binary(image_path):
    files = os.listdir(image_path)
    for file in files:
        os.chdir(image_path)
        img_color  = cv2.imread(file, cv2.IMREAD_COLOR)
        img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        ret, img_binary = cv2.threshold(img_gray, 212, 255, cv2.THRESH_BINARY)
        os.chdir(binary_img_path)
        cv2.imwrite(str(file),img_binary)
        os.chdir('..')
        #경로 초기화

            


        
img_preprocessing_binary(modified_path)

In [14]:
"""
전처리 후 npy파일로 저장하기위한 코드
test_1 행렬 형태 [인덱스, in_height, in_width, color, Rey, target_1dimension]
입력은 이미지 형태로 CNN 모델을 지나고 출력은 1차원 행렬
1차원 행렬을 reshape를 이용하여 변형하여 이미지로 출력하는게 목표
xtrain,xtest,xvalid,xvalid_test,ytrain,ytest 형태로 이용될 예정
일단 사이즈는 강제로 256,256으로 변환시키는 방식
"""
import numpy as np
import re
import os
from PIL import Image
from keras.preprocessing.image import img_to_array
current_path = os.path.abspath('')
input_img_path = os.path.join(current_path,"binary")
target_img_path = os.path.join(current_path,"target_img")
input_files = os.listdir(input_img_path)
target_files = os.listdir(target_img_path)
datasets = []
i = 0

for input_file, target_file in zip(input_files, target_files):
    dataset= []
    input_pro = []
    name = str(input_file)
    #레이놀즈 수 파일 이름에서 추출
    R = float(name[5]+"."+name[6])*10**int(name[7])/10**6
    #인풋 이미지 파일 경로로 들어가서 인풋 이미지 전처리 과정
    os.chdir(input_img_path)
    input_img = Image.open(input_file)
    #이미지 리사이징
    input_img = input_img.resize((256, 256))
    #이미지 nunpy배열로 변환
    input_img_tensor = img_to_array(input_img)
    input_img_tensor = input_img_tensor.reshape(65536).astype('float64')/ 255
    #타겟 이미지 파일 경로로 들어가서 전처리 과정
    os.chdir(target_img_path)
    target_img = Image.open(target_file)
    target_img = target_img.resize((256,256))
    #타겟 이미지는 1차원 배열로 생성
    target_img_tensor = img_to_array(target_img)
    target_img_tensor = target_img_tensor.reshape(196608).astype('float64')/ 255
    dataset.append(input_img_tensor)
    dataset.append(R)
    dataset.append(target_img_tensor)
    datasets.insert(i,dataset)
    i += 1
    os.chdir('..')
datasets = np.array(datasets, dtype = 'object')
np.save("test.npy", datasets)

In [15]:
print(datasets.shape)

(638, 3)


In [7]:
os.chdir("..")